# 电力潮流计算
*Copyright (c) 2023 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*
## 背景介绍

电力潮流计算是电力系统中一类重要的分析计算，其目的是在给定的运行条件下求取电力系统各个节点的电压及功率分布等信息。潮流计算几乎是所有电力系统分析的基础，同时也为电力系统的规划，扩建，和运行方式提供了支撑。一个简单的例子是，对于一个仍在规划中的电力系统，我们通过潮流计算可以检验所提出的电力系统规划方案能否满足各种运行方式的要求。因此电力潮流计算有着非常重要的实际意义。

在潮流模型中，电力系统的各个部分由节点和连接节点的线表示。对于交流电而言，每个节点所需要考虑的参数一般有四个，分别是电压幅度（voltage magnitude），相位(phase angle)，有功功率(active power)和无功功率(reactive power)。其中有功功率指的是电路实际将电能转化为其他形式能量的功率，而无功功率指的是能量在电源和负载之间不断来回跳动产生的功率。根据给定的初始条件，通过解节点电压方程，我们即可得到每个节点上未知的参数。从数学模型角度来看，潮流计算可以规约于解决一组非线性方程。具体地，对于一个拥有 $n$ 个节点的电力系统里的第 $i$ 个节点，我们有：

$$\begin{cases}
P_i-U_i\sum^n_{j=1}U_j(G_{ij}\cos\delta_{ij}+B_{ij}\sin\delta_{ij}) &= 0 \\
Q_i-U_i\sum^n_{j=1}U_j(G_{ij}\sin\delta_{ij}-B_{ij}\cos\delta_{ij}) &= 0 
\end{cases}$$
其中 $P_i$ 表示第 $i$ 个节点的净注入有功功率，$Q_i$ 表示第 $i$ 个节点的净注入无功功率， $U_i$ 表示第 $i$ 个节点的电压。同时，这里 $G$ 表示导纳矩阵的实部， $B$ 表示导纳矩阵的虚部， $\delta_{ij}$ 表示第 $i$ 个和第 $j$ 个节点的相位差。

非线性方程的求解是一个困难的问题，因此我们常常使用一些数值方法去求取近似解。其中比较常用的一种方法是牛顿迭代法（Newton-Raphson method），它可以通过迭代次数的增加，而越来越接近方程的解。简单来说，牛顿迭代法就是从一个初始点出发，通过求解导数来找到合适的优化方向，不断更新我们的近似解，从而得到更加接近真实解的结果。

## 量子解决方案

我们已经知道牛顿迭代法是解决潮流计算的一种常用的经典算法，其解决潮流计算问题的复杂度主要来源于对于雅可比矩阵求逆的过程，这个过程的复杂度为$\mathcal{O}(poly(n))$。在牛顿迭代法中，雅可比矩阵求逆的过程也可以被看作是求解线性方程组的过程。而在量子计算领域，已经有多个量子算法被提出用于解决线性方程组，如Harrow-Hassidim-Lloyd（HHL）算法，变分量子线性求解器（Variational Quantum Linear Solver, VQLS）。并且相比于经典算法，它们已经被证明在一定条件下在解决线性方程组问题上存在着指数加速的优势。在这里，我们可以使用变分量子线性求解器（VQLS）来替换经典牛顿迭代法中复杂度较高的雅可比矩阵求逆过程，从而实现潮流计算问题的量子加速 [1]。我们在量子应用模型库中同样提供了变分量子线性求解器的相关模型和教程，您可以阅读其教程来了解该算法的更多细节。接下来，我们将以一个5节点系统为例，展示基于牛顿迭代法和变分量子线性求解器的潮流计算量子解决方案。

## 使用教程

我们给出了一个设置好参数，可以直接进行电力潮流计算的配置文件。用户只需在`config.toml`里修改相应的参数，并在终端运行
`python power_flow.py --config config.toml`，即可对给定数据进行潮流分析。
### 输出结果
电力潮流的计算结果将被记录到文件 `pf_result.txt` 中。同时我们的优化过程将被记录在日志文件`power_flow.log`中，用户可以看到随着循环数的增加，损失大小的变化。



### 在线演示
这里，我们给出一个在线演示的版本，可以在线进行测试。首先定义配置文件的内容：

In [1]:
test_toml = r"""
# 模型的整体配置文件
# 存储潮流计算初始数据文件的路径。
data_dir = './ieee5cdf.txt'

# 结束潮流计算的误差阈值, 默认为1e-3。
threshold = 1e-3

# 潮流计算中最小迭代次数。
minIter = 3

# 潮流计算中最大迭代次数。
maxIter = 100

# 参数化量子电路的层数。
depth = 4

# 量子电路优化迭代次数。
iterations = 100

# 优化器的学习率。
LR = 0.1

# 参数化量子电路优化中损失函数的阈值。默认为0。
gamma = 0
"""

接下来是模型运行部分的代码：

In [3]:
import os
import warnings

warnings.filterwarnings('ignore')
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import toml
from paddle_quantum.data_analysis.power_flow import data_to_Grid

config = toml.loads(test_toml)
file_name = config.pop('data_dir')
grid = data_to_Grid(file_name)
grid.powerflow(**config)
print("输出电力潮流计算结果:")
grid.printResults()
Error = grid.tolerances[-1] 
print(f"误差为: {Error}")
grid.saveResults()

100%|██████████| 100/100 [01:20<00:00,  1.25it/s]

输出电力潮流计算结果:
Power Flow:

| Bus |    Bus     |    V     |  Angle   |      Injection      |      Generation     |        Load        |
| No  |    Name    |    pu    |  Degree  |     MW   |   MVar   |     MW   |  Mvar    |     MW  |     MVar |
|   1 |Bus 1     HV|    1.060 |    0.000 |  129.816 |   24.447 |  129.816 |   24.447 |   0.000 |    0.000 |
|   2 |Bus 2     HV|    1.036 |   -0.046 |   20.000 |   20.000 |   40.000 |   30.000 |  20.000 |   10.000 |
|   3 |Bus 3     HV|    1.009 |   -0.084 |  -45.000 |  -15.000 |    0.000 |   -0.000 |  45.000 |   15.000 |
|   4 |Bus 4     HV|    1.007 |   -0.090 |  -40.000 |   -5.000 |   -0.000 |    0.000 |  40.000 |    5.000 |
|   5 |Bus 5     HV|    1.002 |   -0.104 |  -60.000 |  -10.000 |    0.000 |   -0.000 |  60.000 |   10.000 |
----------------------------------------------------------------------------------------------------------

Network and losses:

|  From |    To |     P     |     Q     |  From |    To |     P     |     Q     |       Br

这里误差指的是求解方程后方程组的绝对误差的最大值。

### 注意事项

这里模型的输入数据需要使用IEEE的通用数据格式 [2]。


## 引用信息

```tex
@article{liu2022quantum,
  title={Quantum Power Flows: From Theory to Practice},
  author={Liu, Junyu and Zheng, Han and Hanada, Masanori and Setia, Kanav and Wu, Dan},
  journal={arXiv preprint arXiv:2211.05728},
  year={2022}
}

@article{pierce1973common,
  title={Common format for exchange of solved load flow data},
  author={Pierce, HE and others},
  journal={IEEE Transactions on Power Apparatus and Systems},
  volume={92},
  number={6},
  pages={1916--1925},
  year={1973}
}
```

## 参考文献
[1] Liu, Junyu, et al. "Quantum Power Flows: From Theory to Practice." arXiv preprint arXiv:2211.05728 (2022).

[2] Pierce, H. E. "Common format for exchange of solved load flow data." IEEE Transactions on Power Apparatus and Systems 92.6 (1973): 1916-1925.